## Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import  NoSuchElementException,ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import os
import re

## Cấu hình tùy chọn cho Edge

In [2]:
options = Options()
options.add_argument("--start-maximized")  # Mở ở chế độ toàn màn hình
options.add_argument("--inprivate")  # Chế độ ẩn danh (InPrivate mode)

## Đăng nhập

In [9]:
driver = webdriver.Edge(options=options)
driver.get("https://www.facebook.com/groups/ued.confessions?locale=vi_VN")
# try:
#     username = driver.find_element(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .xod5an3 input')
#     password = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg input')[0]
#     login = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg')[1]
# except:
#     username = driver.find_element(By.CSS_SELECTOR, 'input[name = "email" and type = "email"]')
#     password = driver.find_element(By.CSS_SELECTOR, 'input[name = "pass" and type = "password"]')
#     login = driver.find_element(By.CSS_SELECTOR, 'div[aria-label = "Đăng nhập"]') 

# sleep(2) 

# username.send_keys('khoalolriot@gmail.com')
# password.send_keys('Akhoa@6204')
# login.click()

## Hàm scroll

In [4]:
def scroll():
    driver.execute_script("window.scrollTo({top: document.body.scrollHeight, behavior: 'smooth'});")
    sleep(1)
def scroll_to_post(post, time):
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", post)
    sleep(time)

## Đường dẫn lưu file

In [5]:
path_normal = 'D:/kpw/crawlFB/post_facebook.csv'
path_reel = 'D:/kpw/crawlFB/post_reel.csv'

## Lưu kết quả

In [6]:
def saveOrCreate(result_post_normal, result_post_reel):
    # Kiểm tra và tạo file CSV cho post_facebook nếu không tồn tại
    if not os.path.exists(path_normal):
        os.makedirs(os.path.dirname(path_normal), exist_ok=True)
        df_normal = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'role', 'time', 'content', 'content_img', 'react_list', 'comment'])
        df_normal.to_csv(path_normal, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_facebook.csv
    if result_post_normal:
        new_post_normal = pd.DataFrame(result_post_normal)
        df_normal = pd.read_csv(path_normal)
        df_normal = pd.concat([df_normal, new_post_normal], ignore_index=True)
        df_normal.to_csv(path_normal, index=False)

    # Kiểm tra và tạo file CSV cho post_reel nếu không tồn tại
    if not os.path.exists(path_reel):
        os.makedirs(os.path.dirname(path_reel), exist_ok=True)
        df_reel = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'time', 'content', 'linkVideo', 'typeVideo'])
        df_reel.to_csv(path_reel, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_reel.csv
    if result_post_reel:
        new_post_reel = pd.DataFrame(result_post_reel)
        df_reel = pd.read_csv(path_reel)
        df_reel = pd.concat([df_reel, new_post_reel], ignore_index=True)
        df_reel.to_csv(path_reel, index=False)


## Lấy ra post đầu tiên của lần crawl trước

In [7]:
def get_first_post():
    # Đọc dữ liệu từ các file CSV
    df_normal = pd.read_csv(path_normal)
    df_reel = pd.read_csv(path_reel)

    # Kiểm tra nếu cả hai dataframe đều rỗng
    if df_normal.empty and df_reel.empty:
        return None
    
    # Nếu df_normal không rỗng
    if not df_normal.empty:
        first_row_normal = df_normal.iloc[0]
    else:
        first_row_normal = None

    # Nếu df_reel không rỗng
    if not df_reel.empty:
        first_row_reel = df_reel.iloc[0]
    else:
        first_row_reel = None

    # So sánh và trả về dòng có index nhỏ hơn nếu cả hai dataframe không rỗng
    if first_row_normal is not None and first_row_reel is not None:
        if first_row_normal['index'] > first_row_reel['index']:
            result = first_row_reel
        else:
            result = first_row_normal
    elif first_row_normal is not None:
        result = first_row_normal
    elif first_row_reel is not None:
        result = first_row_reel
    else:
        result = None

    return result


## List kết quả

In [28]:
result_post_normal = []
result_post_reel = []

## Main

In [ ]:
saveOrCreate(None, None) # Tạo csv

index = 0
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
prev_result = get_first_post()
while True: 
    while True:
        # Lưu kết quả sau mỗi vòng lặp
        post_data_normal = {}
        post_data_reel = {} 
        
        # Tắt cảnh báo
        try:
            close = driver.find_element(By.XPATH,'//*[@id="facebook"]/body/div[6]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[3]')
            close.click()
            sleep(5)
        except NoSuchElementException: 
            pass  
        
        # Scroll mỗi khi crawl hết trong danh sách có sẵn
        if index >= len(post_list):  
            scroll()
            post_list_new = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
            if len(post_list_new) == len(post_list):
                print()
                print('Thu thập được:',len(post_list),'bài đăng')
                break 
            else:
                post_list = post_list_new 
        
        # Main
        try: # Post normal 
            # Scroll to post
            scroll_to_post(post_list[index], 1)
            print(f"Đã scroll tới bài đăng số {index + 1}:")
            print('post normal')
            
            # Lấy Xpath của bài post 
            main  = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
            
            # Chia nhỏ Xpath thành 3 phần
            info_box  = main.find_element(By.XPATH,'./div[2]')            
            content_box = main.find_element(By.XPATH,'./div[3]')
            react_cmt_box = main.find_element(By.XPATH,'./div[4]')
            
            # Lấy thông tin người đăng
            user_link = info_box.find_element(By.XPATH, './div/div[2]/div/div[2]').find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            
            user_name = info_box.find_element(By.XPATH, './div/div[2]/div/div[1]').find_element(By.CSS_SELECTOR, '.xt0psk2').text  
            
            time_post = info_box.find_element(By.XPATH, './div/div[2]/div/div[2]')

            try: 
                role = time_post.find_element(By.CSS_SELECTOR,'.x3nfvp2.x1kgmq87').text
            except NoSuchElementException:
                role = None

            try: 
                time_link = time_post.find_element(By.CSS_SELECTOR, 'use').get_attribute('xlink:href')
                time_post = driver.find_element(By.CSS_SELECTOR, f'text{time_link}').get_attribute('textContent')
            except NoSuchElementException:  
                time_post = time_post.text.replace('·','').strip()

            # Lấy thông tin bài đăng
            try: # Có nội dung
                main_content = content_box.find_element(By.XPATH, './div[1]')
                try: 
                    more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
                    if more_content.text.strip() == 'Xem thêm': 
                        try:
                            scroll_to_post(more_content, 0)
                            more_content.click()
                        except ElementClickInterceptedException: 
                            driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
                            sleep(1)
                            more_content.click()
                except NoSuchElementException: 
                    pass
                main_content = content_box.find_element(By.XPATH, './div[1]').text
            except NoSuchElementException: # Không có nội dung
                main_content = None

            # Lấy ảnh trong bài đăng
            content_img_post = {}        
            try: # Có ảnh  
                content_img = content_box.find_element(By.XPATH, './div[2]')
                content_img_list = content_img.find_elements(By.CSS_SELECTOR, 'img')
                for i, img in enumerate(content_img_list):
                    content_img_post[i] = img.get_attribute('src')
            except NoSuchElementException: # Không có ảnh
                pass

            # Lấy thông tin lượt người thả cảm xúc
            react_list = {
                "Thích" : 0,
                'Yêu thích' : 0,
                'Thương thương': 0,
                'Haha': 0,
                'Wow': 0,
                'Buồn': 0,
                'Phẫn nộ': 0
            }
            try: # Bài đăng có người thả cảm xúc
                react = react_cmt_box.find_element(By.XPATH,'./div/div/div[1]')

                total_cmt = react.find_element(By.XPATH,'./div/div[1]/div/div[2]').text

                total_react = react.find_element(By.XPATH,'./div/div[1]/div/div[1]/div') 

                react_icon = react.find_elements(By.XPATH,'./div/div[1]/div/div[1]/span/span/span')
                for react in react_icon: 
                    rs = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[0].strip()
                    for i in react_list: 
                        if i== rs:
                            react_list[i] = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[1].strip().split()[0]
            except NoSuchElementException: # Bài đăng không có người thả cảm xúc
                pass
            
            # Lấy thông tin comment của bài đăng
            post_data_cmt = []
            react_cmt_box = main.find_element(By.XPATH,'./div[4]')

            try: # Bài dăng có nhiều comment 
                # Click vào xem thêm
                more_cmt = react_cmt_box.find_element(By.XPATH, './div/div/div[@class="xdj266r xat24cr x1d52u69 xktsk01"]')
                if 'Xem thêm' in more_cmt.text:
                    try:
                        scroll_to_post(more_cmt, 0)
                        more_cmt.click()
                    except: 
                        driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
                        more_cmt.click()
                        
                sleep(3)
                
                # Lấy ra Xpath của khung chứa nội dung comment
                cmt_box = driver.find_element(By.CSS_SELECTOR, 'div.x1mh8g0r.xh8yej3 div.xkhd6sd.x1gslohp')

                scroll_to_post(cmt_box, 1)
                
                # Lấy ra danh sách các comment
                list_cmt = cmt_box.find_elements(By.XPATH, "./div[position() < last()]")
                
                # Lặp qua danh sách các comment
                for cmt in list_cmt:
                    scroll_to_post(cmt, 0)
                    
                    # Nhấn tất cả các nút xem thêm của comment đó
                    while True:
                        try:
                            more_cmt_button_list = cmt.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
                            has_clicked = False 
                            for button in more_cmt_button_list:
                                if "Xem" in button.text and "phản hồi" in button.text:
                                    try:
                                        button.click()
                                    except:
                                        scroll_to_post(button, 0)
                                        button.click()
                                    sleep(2)
                                    has_clicked = True
                                    break  
                            if not has_clicked:
                                break
                        except Exception:
                            break
                        
                    content = cmt.text.split('\n')
                    results = []
                    # Tiền xử lí comment
                    try:
                        i = 0
                        for index_, line in enumerate(content):  
                            if line == 'Thích': 
                                cmt_sub = content[i: index_ - 1]
                                results.append(cmt_sub)
                            if line == "Chia sẻ" and content[index_ + 1] != 'Đã chỉnh sửa':
                                i = index_ + 1

                            if line == 'Đã chỉnh sửa' and content[index_ + 1].isdigit():
                                i = index_ + 2
                            elif line == 'Đã chỉnh sửa' and re.search(r'[a-zA-Z]', content[index_ + 1]):
                                i = index_ + 1 
                            if line == 'Viết phản hồi công khai…':
                                i = index_ + 1        
                    except: pass  
                    
                    # Lưu người viết và nội dung comment 
                    for result in results: 
                        for index_, line in enumerate(result):
                            if index_ == 0: 
                                if line.isdigit():
                                    name = result[index_ + 1]
                                else: 
                                    name = result[index_]
                                    num = index_
                            content = ' '.join(result[num + 1: ])
                            if line == '  ·' and result[index_ + 1] == 'Theo dõi':
                                content = ' '.join(result[index_ + 2 :])
                                break
                            
                        comment = {
                            'name' : name, 
                            'content': content
                        }
                        
                        post_data_cmt.append(comment)
                
                # Tắt pop-up comment       
                close = driver.find_element(By.CSS_SELECTOR, 'div[role="button"][aria-label="Đóng"]')
                close.click()
                
            except NoSuchElementException: # Bài đăng ít comment 
                # Lấy ra khung chứa comment
                cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div')
                
                if len(cmt_box) < 3: # Không có comment nào trong bài đăng
                    pass 
                else: # Có comment trong bài đăng
                    # Lấy ra khung chứa comment
                    cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div[position() > 2 and position() < last()]')
                    
                    # Lặp qua danh sách các comment
                    for cmt in cmt_box:
                        # Lấy ra người viết và nội dung
                        name = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/span[1]').text
                        content = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/div').text
                        
                        # Lưu thông tin
                        comment = {
                            'name' : name, 
                            'content': content
                        }
                        post_data_cmt.append(comment)
                # Lấy ra khung chứa comment
                cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div')
                
                if len(cmt_box) < 3: # Không có comment nào trong bài đăng
                    pass 
                else: # Có comment trong bài đăng
                    # Lấy ra khung chứa comment
                    cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div[position() > 2 and position() < last()]')
                    
                    # Lặp qua danh sách các comment
                    for cmt in cmt_box:
                        # Lấy ra người viết và nội dung
                        name = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/span[1]').text
                        content = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/div').text
                        
                        # Lưu thông tin
                        comment = {
                            'name' : name, 
                            'content': content
                        }
                        post_data_cmt.append(comment)
            
            # Lưu thông tin toàn bộ bài post 
            post_data_normal = {
                'index': index + 1,
                'linkPoster': user_link if user_name != 'Người tham gia ẩn danh' else 'No link',
                'name': user_name,
                'role': role if role else 'Thành viên',
                'time': time_post,
                'content': main_content if main_content else 'No content',
                'content_img': content_img_post if content_img_post else 'No picture',
                'react_list': react_list,
                'comment': post_data_cmt if post_data_cmt else 'No comment'
            }
            print(post_data_normal)
            index += 1
        except NoSuchElementException: # video
            print(f"Đã scroll tới bài đăng số {index + 1}:")
            print('post video')
            
            # Lấy ra Xpath cuả bài post
            main = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
            content_box = main.find_element(By.XPATH,'./div[2]')
            
            # Thông tin bài post
            link = content_box.find_element(By.CSS_SELECTOR, 'a')
            href = link.get_attribute('href')
            
            box = link.find_element(By.XPATH,'./div[1]/div[3]/div/div/div[1]/div/div/div/div/div/div[2]/span/span')
            
            linkPoster = box.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
            
            box = box.text.split('·')
            typeVideo = box[0].replace('\n', '').strip()
            name = box[1].replace('\n', '').strip()
            time = box[2].replace('\n', '').strip()
            
            # Lấy ra nội dung bài post
            try: # Có nội dung
                main_content = link.find_element(By.XPATH,'./div[1]/div[2]/div/div/div[2]/span/div')
                try: 
                    more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
                    if more_content.text.strip() == 'Xem thêm': 
                        scroll_to_post(more_content, 0)
                        more_content.click()
                except NoSuchElementException: 
                    pass
                main_content = link.find_element(By.XPATH, './div[1]/div[2]/div/div/div[2]/span/div').text
            except NoSuchElementException: # Không có nội dung
                main_content = None
            
            # Lưu thông tin bài post
            post_data_reel = {
                'index': index + 1,
                'linkPoster': linkPoster,
                'name': name,
                'time': time,
                'content': main_content if main_content else 'No content',
                'linkVideo': href,
                'typeVideo': typeVideo,
            }
            
            index += 1
            print(post_data_reel)
        
        # Kiểm tra trước đó đã crawl url này hay chưa
        if prev_result is None: # Chưa crawl
            pass
        else: # Đã crawl
            if prev_result == post_data_normal: #Tới được bài đăng đầu tiên của đợt crawl trước
                break
            if prev_result == post_data_reel: #Tới được bài đăng đầu tiên của đợt crawl trước
                break
        # Thêm kết quả vào danh sách
        if post_data_normal:  
            result_post_normal.append(post_data_normal)
        if post_data_reel:
            result_post_reel.append(post_data_reel)
        sleep(1)
    try: 
        # Tắt pop-up comment       
        close = driver.find_element(By.CSS_SELECTOR, 'div[role="button"][aria-label="Đóng"]')
        close.click()
    except:
        pass
    scroll()
    if index == len(post_list): 
        break 
print('Crawl thành công')


Đã scroll tới bài đăng số 38:
post normal
{'index': 38, 'linkPoster': 'No link', 'name': 'Người tham gia ẩn danh', 'role': 'Thành viên', 'time': '13 tháng 11 lúc 23:51', 'content': 'What ???? Ngáo ngơ được vô chung kết ???? Đúng là ban văn nghệ :)) sắp xếp hết rồi', 'content_img': 'No picture', 'react_list': {'Thích': '17', 'Yêu thích': 0, 'Thương thương': 0, 'Haha': '28', 'Wow': 0, 'Buồn': '5', 'Phẫn nộ': 0}, 'comment': [{'name': 'An Nguyễn', 'content': 'Mình xin giải đáp thắc mắc của bạn, tiết mục của BVN được vô chung kết chỉ để diễn và không có tư cách tham gia dự thi như các tiết mục của lớp. Việc có 2 tiết mục trong chung kết là để đảm bảo khoảng cách thời gian giữa các lớp, 2 tiết mục này để hỗ trợ diễn cho Khoa thôi. Hi vọng các thắc mắc nào bạn nên làm rõ trước khi hỏi trên này nhé! Mình cũng mong các bạn tham gia cuộc thi với tinh thần thỏa mái và tránh có những thông tin không tốt vì mục đích cuối cùng là để tri ân cho thầy cô nhé.'}, {'name': 'Nguyễn Gia Huy', 'content': 'H

KeyboardInterrupt: 

In [ ]:
saveOrCreate(result_post_normal, result_post_reel)
print('Save thành công')

In [22]:
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
index =  3

main  = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
scroll_to_post(main, 0)

sleep(2)

# Lấy thông tin comment của bài đăng
post_data_cmt = []
react_cmt_box = main.find_element(By.XPATH,'./div[4]')

try: # Bài dăng có nhiều comment 
    # Click vào xem thêm
    more_cmt = react_cmt_box.find_element(By.XPATH, './div/div/div[@class="xdj266r xat24cr x1d52u69 xktsk01"]')
    if 'Xem thêm' in more_cmt.text:
        try:
            scroll_to_post(more_cmt, 0)
            more_cmt.click()
        except: 
            driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
            more_cmt.click()
            
    sleep(2)
    
    # Lấy ra Xpath của khung chứa nội dung comment
    cmt_box = driver.find_element(By.CSS_SELECTOR, 'div.xkhd6sd.x1gslohp')

    scroll_to_post(cmt_box, 1)
    
    # Lấy ra danh sách các comment
    list_cmt = cmt_box.find_elements(By.XPATH, "./div[position() < last()]")
    
    # Lặp qua danh sách các comment
    for cmt in list_cmt:
        scroll_to_post(cmt, 0)
        
        # Nhấn tất cả các nút xem thêm của comment đó
        while True:
            try:
                more_cmt_button_list = cmt.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
                has_clicked = False 
                for button in more_cmt_button_list:
                    if "Xem" in button.text and "phản hồi" in button.text:
                        try:
                            button.click()
                        except:
                            scroll_to_post(button, 0)
                            button.click()
                        sleep(2)
                        has_clicked = True
                        break  
                if not has_clicked:
                    break
            except Exception:
                break
            
        content = cmt.text.split('\n')
        results = []
        # Tiền xử lí comment
        try:
            i = 0
            for index_, line in enumerate(content):  
                if line == 'Thích': 
                    cmt_sub = content[i: index_ - 1]
                    results.append(cmt_sub)
                if line == "Chia sẻ" and content[index_ + 1] != 'Đã chỉnh sửa':
                    i = index_ + 1

                if line == 'Đã chỉnh sửa' and content[index_ + 1].isdigit():
                    i = index_ + 2
                elif line == 'Đã chỉnh sửa' and re.search(r'[a-zA-Z]', content[index_ + 1]):
                    i = index_ + 1 
                if line == 'Viết phản hồi công khai…':
                    i = index_ + 1        
        except: pass  
        
        # Lưu người viết và nội dung comment 
        for result in results: 
            for index_, line in enumerate(result):
                if index_ == 0: 
                    if line.isdigit():
                        name = result[index_ + 1]
                    else: 
                        name = result[index_]
                        num = index_
                content = ' '.join(result[num + 1: ])
                if line == '  ·' and result[index_ + 1] == 'Theo dõi':
                    content = ' '.join(result[index_ + 2 :])
                    break
                
            comment = {
                'name' : name, 
                'content': content
            }
            
            post_data_cmt.append(comment)
    
    # Tắt pop-up comment       
    close = driver.find_element(By.CSS_SELECTOR, 'div[role="button"][aria-label="Đóng"]')
    close.click()
    
except NoSuchElementException: # Bài đăng ít comment 
    # Lấy ra khung chứa comment
    cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div')
    
    if len(cmt_box) < 3: # Không có comment nào trong bài đăng
        pass 
    else: # Có comment trong bài đăng
        # Lấy ra khung chứa comment
        cmt_box = react_cmt_box.find_elements(By.XPATH, './div/div/div[position() > 2 and position() < last()]')
        
        # Lặp qua danh sách các comment
        for cmt in cmt_box:
            # Lấy ra người viết và nội dung
            name = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/span[1]').text
            content = cmt.find_element(By.XPATH, './div/div/div[2]/div[2]/div[1]/div/div/div').text
            
            # Lưu thông tin
            comment = {
                'name' : name, 
                'content': content
            }
            post_data_cmt.append(comment)
print(post_data_cmt)


[{'name': 'Light Of Night', 'content': 'Cưới chồng là chồng nuôi, ở nhà chăm con. Đúng ngành đúng nghề'}, {'name': 'Lê Ngọc Linh', 'content': 'omg :)) h sao ta'}]
